In [ ]:
import numpy as np
import pandas as pd

In [ ]:
interactions = pd.read_csv('RAW_interactions.csv')
recipes = pd.read_csv('RAW_recipes.csv')

In [ ]:
interactions.head()

In [ ]:
recipes.head()

In [ ]:
recipes=recipes[['name','id','tags','contributor_id','ingredients']]

In [ ]:
recipes.head()

In [ ]:
interactions=interactions[['user_id','rating','recipe_id']]

In [ ]:
interactions.head()

In [ ]:
recipes.rename(columns={'id':'recipe_id'},inplace=True)

In [ ]:
recipes

In [ ]:
merge_df = recipes.merge(interactions,on='recipe_id')

In [ ]:
merge_df

In [ ]:
merge_df.isnull().sum()

In [ ]:
merge_df.dropna(inplace=True)
collab_merge_df=merge_df
popularity_merge_df=merge_df

In [ ]:
merge_df.duplicated().sum()

In [ ]:
import ast

In [ ]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i) 
    return L 

In [ ]:
temp_df=merge_df

In [ ]:
temp_df['ingredients'] = temp_df['ingredients'].apply(convert)

In [ ]:
temp_df['tags'] = temp_df['tags'].apply(convert)

In [ ]:
def convert4(L):
    L1 = []
    for i in L:
        L1.append(i.replace("-",""))
    return L1

def convert5(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
temp_df['ingredients'] = temp_df['ingredients'].apply(convert4)
temp_df['tags'] = temp_df['tags'].apply(convert4)
temp_df['ingredients'] = temp_df['ingredients'].apply(convert5)
temp_df['tags'] = temp_df['tags'].apply(convert5)

In [ ]:
temp1_df=temp_df[['name','recipe_id','tags','ingredients','rating']]

In [ ]:
temp1_df

In [ ]:
temp3_df = temp1_df.groupby('recipe_id').count()['rating'].reset_index()

In [ ]:
temp3_df.rename(columns={'rating':'num_rating'},inplace=True)

In [ ]:
temp3_df

In [ ]:
temp4_df=temp3_df.merge(temp1_df,on='recipe_id')

In [ ]:
temp4_df=temp4_df[['recipe_id','num_rating','name','tags','ingredients']]

In [ ]:
temp4_df

In [ ]:
temp4_df=temp4_df.drop_duplicates('recipe_id')

In [ ]:
temp4_df

In [ ]:
merge1_df=temp4_df

In [ ]:
merge1_df['fin_tags']=merge1_df['tags']+merge1_df['ingredients']

In [ ]:
merge1_df=merge1_df[['name','recipe_id','fin_tags','num_rating']]

In [ ]:
merge1_df

In [ ]:
merge1_df['fin_tags'][231636]


In [ ]:
merge1_df['fin_tags'][0]

In [ ]:
merge1_df['fin_tags'] = merge1_df['fin_tags'].apply(lambda x: " ".join(x))

In [ ]:
merge1_df

In [ ]:
merge2_df=merge1_df

In [ ]:
merge2_df

In [ ]:
merge3_df = merge2_df[merge2_df['num_rating']>=10].sort_values('recipe_id',ascending=False).reset_index()

In [ ]:
merge3_df=merge3_df.drop_duplicates('recipe_id')


In [ ]:
merge3_df=merge3_df[['name','recipe_id','fin_tags','num_rating']]

In [ ]:
merge3_df.rename(columns={'name':'recipe_name'},inplace=True)

In [ ]:
merge3_df

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=500,stop_words='english')

In [ ]:
vector = cv.fit_transform(merge3_df['fin_tags']).toarray()

In [ ]:
vector

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
def recommend(dish):
    recommended_products=[];
    if not merge3_df[merge3_df['recipe_name'] == dish].empty:
        index = merge3_df[merge3_df['recipe_name'] == dish].index[0]
        distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
        for i in distances[1:10001]:
            recommended_products.append(merge3_df.iloc[i[0]].recipe_id)
        return recommended_products

In [ ]:
merge3_df

In [ ]:
pip install fuzzywuzzy

In [ ]:
pip install python-Levenshtein

In [ ]:
names_df=merge3_df[['recipe_name']]

In [ ]:
from fuzzywuzzy import fuzz, process

In [ ]:
target_name = 'pasta'

In [ ]:
similar_names = process.extract(target_name, names_df['recipe_name'], limit=3)

In [ ]:
recommended_products=[]
for name in similar_names:
    if name[1]/name[2]>=0.1:
        recommended_products=recommended_products+recommend(name[0])

In [ ]:
recommended_df = pd.DataFrame(recommended_products, columns=['recipe_id'])
recommended_df

In [ ]:
recommended_df=recommended_df.drop_duplicates('recipe_id').reset_index()

In [ ]:
recommended_df

In [ ]:
recommended_df=recommended_df[['recipe_id']]

In [ ]:
recommended_df

In [ ]:
merge_df=merge_df[['name','recipe_id','user_id','rating']]


In [ ]:
merge_df

In [ ]:
collab_df=recommended_df.merge(merge_df,on='recipe_id')

In [ ]:
collab_df

In [ ]:
collab_df.isnull().sum()

In [ ]:
collab_merge_df=collab_df[['name','recipe_id','user_id','rating']]
collab_final_df=collab_merge_df

In [ ]:
collab_merge_df

In [ ]:
x=collab_merge_df.groupby('user_id').count()['rating'] > 10
x

In [ ]:
users_with_more_ratings=x[x].index
users_with_more_ratings

In [ ]:
filtered_rating = collab_merge_df[collab_merge_df['user_id'].isin(users_with_more_ratings)]
filtered_rating

In [ ]:
y = filtered_rating.groupby('name').count()['rating']>=20
famous_food = y[y].index
famous_food

In [ ]:
final_ratings = filtered_rating[filtered_rating['name'].isin(famous_food)]
final_ratings

In [ ]:
pt = final_ratings.pivot_table(index='name',columns='user_id',values='rating')
pt.fillna(0,inplace=True)
pt

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity_scores = cosine_similarity(pt)

In [ ]:
similarity_scores.shape

In [ ]:
def recommend_collab(dish_name):
    similar_name = process.extract(dish_name, final_ratings['name'], limit=1)
    index = np.where(pt.index==similar_name[0][0])[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:21]
    
    data = []
    for i in similar_items:
        data.append(pt.index[i[0]])
    return data

In [ ]:
collab_similar_items=recommend_collab('pasta')

In [ ]:
collab_final_df=collab_merge_df[collab_merge_df['name'].isin(collab_similar_items)]
collab_final_df

In [ ]:
content_final_df=collab_df[['name','recipe_id']]

In [ ]:
content_final_df=content_final_df.drop_duplicates('recipe_id').reset_index()

In [ ]:
content_final_df=content_final_df[['name','recipe_id']]

In [ ]:
content_final_df=content_final_df.head(20)

In [ ]:
content_final_df

In [ ]:
collab_final_df=collab_final_df[['name','recipe_id']]

In [ ]:
collab_final_df=collab_final_df.drop_duplicates('recipe_id').reset_index()

In [ ]:
collab_final_df=collab_final_df[['name','recipe_id']]

In [ ]:
collab_final_df=collab_final_df.head(20)

In [ ]:
collab_final_df

In [ ]:
num_ratings_df=popularity_merge_df.groupby('recipe_id').count()['rating'].reset_index()

In [ ]:
num_ratings_df

In [ ]:
popularity_merge_df=collab_df

In [ ]:
avg_rating_df=collab_df.groupby('recipe_id').mean()['rating'].reset_index()

In [ ]:
avg_rating_df

In [ ]:
num_ratings_df.rename(columns={'rating':'num_ratings'},inplace=True)

In [ ]:
num_ratings_df

In [ ]:
avg_rating_df.rename(columns={'rating':'avg_rating'},inplace=True)

In [ ]:
avg_rating_df

In [ ]:
popular_df=num_ratings_df.merge(avg_rating_df,on='recipe_id')

In [ ]:
popular_df

In [ ]:
popular_df = popular_df[popular_df['num_ratings']>=20].sort_values('avg_rating',ascending=False)

In [ ]:
popular_df

In [ ]:
popular_df = popular_df.merge(collab_df,on='recipe_id').drop_duplicates('recipe_id')[['recipe_id','name','num_ratings','avg_rating']].sort_values(by=["avg_rating", "num_ratings"], ascending=[False, False])

In [ ]:
popular_df

In [ ]:
popularity_final_df=popular_df[['name','recipe_id']]

In [ ]:
popularity_final_df=popularity_final_df.drop_duplicates('recipe_id').reset_index()

In [ ]:
popularity_final_df=popularity_final_df[['name','recipe_id']]

In [ ]:
popularity_final_df=popularity_final_df.head(20)

In [ ]:
popularity_final_df

In [ ]:
merged_final_df = pd.concat([content_final_df, collab_final_df], ignore_index=True)
merged_final_df = pd.concat([merged_final_df,popularity_final_df], ignore_index=True)

In [ ]:
merged_final_df.drop_duplicates('recipe_id')

In [ ]:
merged_final_df['weight']=0

In [ ]:
merged_final_df

In [ ]:
content_final_df

In [ ]:
def assign_weights():
    for idx,row in merged_final_df.iterrows():
        wght=0
        idx1 = content_final_df[content_final_df['recipe_id'] == row[1]].index[0] if any(content_final_df['recipe_id'] == row[1]) else 20
        idx2= collab_final_df[collab_final_df['recipe_id'] == row[1]].index[0] if any(collab_final_df['recipe_id'] == row[1]) else 20
        idx3 = popularity_final_df[popularity_final_df['recipe_id'] == row[1]].index[0] if any(popularity_final_df['recipe_id'] == row[1]) else 20
        wght=0.5*(20-idx1)+0.3*(20-idx2)+0.2*(20-idx3)
        row_idx=merged_final_df[merged_final_df['recipe_id'] == row[1]].index[0]
        merged_final_df.at[row_idx, 'weight'] = wght

In [ ]:
assign_weights()

In [ ]:
merged_final_df

In [ ]:
final_system = merged_final_df.sort_values(by='weight', ascending=False).reset_index()
final_system=final_system[['name','recipe_id']].head(20)

In [ ]:
final_system